# Make new DataFrame in which the data are ordered and are reported with their absolute salinity and conservative temperature

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import gsw

In [2]:
df = pd.read_excel('../../data/ISA_CTD/ctd_isa_all.xlsx',skiprows=1)
df.rename(columns={'Pressure (db)': 'Pressure', 'Salinity (psu)': 'Salinity', 'Temperature (degC)': 'Temperature'},
         inplace=True)
df

,Cruise,Station,Type,Year,Month,Day,Hour,Minute,Second,Latitude,...,Bot. Depth (m),Pressure,Salinity,Temperature,Conductivity (S/m),Ship,Data Owner,File name,Citation,Instrument
0,UNIS AB 2011 - Isfjorden/Billefjorden,ISA,C,2011,12,14,12,5,34,78.257,...,80.0,1,34.3150,0.8435,NaN,KV Svalbard/Viking Explorer/Polar Circle/Snow ...,UNIS,ISA_14122011.txt,UNIS Hydrographic Database,SAIV
1,UNIS AB 2011 - Isfjorden/Billefjorden,ISA,C,2011,12,14,12,5,34,78.257,...,80.0,2,34.2900,0.8440,NaN,KV Svalbard/Viking Explorer/Polar Circle/Snow ...,UNIS,ISA_14122011.txt,UNIS Hydrographic Database,SAIV
2,UNIS AB 2011 - Isfjorden/Billefjorden,ISA,C,2011,12,14,12,5,34,78.257,...,80.0,3,34.3000,0.8415,NaN,KV Svalbard/Viking Explorer/Polar Circle/Snow ...,UNIS,ISA_14122011.txt,UNIS Hydrographic Database,SAIV
3,UNIS AB 2011 - Isfjorden/Billefjorden,ISA,C,2011,12,14,12,5,34,78.257,...,80.0,4,34.3100,0.8410,NaN,KV Svalbard/Viking Explorer/Polar Circle/Snow ...,UNIS,ISA_14122011.txt,UNIS Hydrographic Database,SAIV
4,UNIS AB 2011 - Isfjorden/Billefjorden,ISA,C,2011,12,14,12,5,34,78.257,...,80.0,5,34.2950,0.8440,NaN,KV Svalbard/Viking Explorer/Polar Circle/Snow ...,UNIS,ISA_14122011.txt,UNIS Hydrographic Database,SAIV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18316,Nansen Legacy KH2018709,94,C,2018,9,12,19,15,48,78.257,...,93.0,61,34.5318,3.4711,3.1696,Kronprins Haakon,Nansen Legacy,sta0094_bin1db.cnv,UNIS Hydrographic Database,SBE911+
18317,Nansen Legacy KH2018709,94,C,2018,9,12,19,15,48,78.257,...,93.0,62,34.5321,3.4686,3.1694,Kronprins Haakon,Nansen Legacy,sta0094_bin1db.cnv,UNIS Hydrographic Database,SBE911+
18318,Nansen Legacy KH2018709,94,C,2018,9,12,19,15,48,78.257,...,93.0,63,34.5327,3.4672,3.1694,Kronprins Haakon,Nansen Legacy,sta0094_bin1db.cnv,UNIS Hydrographic Database,SBE911+
18319,Nansen Legacy KH2018709,94,C,2018,9,12,19,15,48,78.257,...,93.0,64,34.5355,3.4638,3.1694,Kronprins Haakon,Nansen Legacy,sta0094_bin1db.cnv,UNIS Hydrographic Database,SBE911+


In [3]:
# Add full date as datetime
df['Date'] = df.apply(lambda row: datetime(int(row.Year),int(row.Month),int(row.Day),
                                           int(row.Hour),int(row.Minute),int(row.Second)),axis=1)

# Add time as number of seconds passed since the beginning
time_start = df['Date'][0]
df['Time'] = df.apply(lambda row: (row.Date-time_start).total_seconds(),axis=1)

# Add depth in m
#df['Depth'] = df.apply(lambda row: -1*gsw.z_from_p(row.Pressure,row.Latitude),axis=1)

# Add absolute salinity
df['SA'] = df.apply(lambda row: gsw.SA_from_SP(row.Salinity,row.Pressure,row.Longitude,row.Latitude),axis=1)

# Add conservative temperature
df['CT'] = df.apply(lambda row: gsw.CT_from_t(row.SA,row.Temperature,row.Pressure),axis=1)

In [4]:
# Drop other columns
df = df.drop(['Cruise', 'Station', 'Type', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second',
       'Bot. Depth (m)', 'Salinity', 'Temperature', 'Conductivity (S/m)',
       'Ship', 'Data Owner', 'File name', 'Citation', 'Instrument',],axis=1)

In [5]:
df.to_csv('../../data/ISA_CTD/ctd_isa_all_processed.csv')